In [1]:
import numpy as np
import nibabel as nib
import pandas as pd
from config_local import helpers


hp = helpers()

In [2]:
_data_path = hp.get_data_path()
print(_data_path)

data/


In [3]:
phenotypic_df = pd.read_csv(_data_path + "NYU_phenotypic.csv").fillna(-999)
dic = {}

for i, row in phenotypic_df.iterrows():
    
    id = str(row['ScanDir ID']).zfill(7)
    fold = int(row['Fold'])
    part = row['Partition']

    if fold != -999:
        file = _data_path + f"{part}/fold{fold}/wmean_mrda{id}_session_1_rest_1.nii.gz"
        img = nib.load(file)
        data = img.get_fdata()
        y = row['DX']
        y_bin = int(y>=1)
        dic[id] = {"img":img, "data":data, "dx":y, "dx_bin":y_bin, "part":part, "fold":fold, "data_reduced":data[10:-10, 10:-10, :-20]}

In [4]:
def train_loader(dic, i, is_reshape=True, pca=False):
    X_val = []
    X_train = []
    y_val = []
    y_train = []

    for id, subj in dic.items():

        if ((subj["part"]=="train") & (subj["fold"]!=i)):
            if is_reshape:
                X_train.append(subj['data_reduced'].reshape(-1))
            else:
                X_train.append(subj['data'])
            y_train.append(subj['dx_bin'])
                
        if ((subj["part"]=="train") & (subj["fold"]==i)):
            if is_reshape:
                X_val.append(subj['data_reduced'].reshape(-1))
            else:
                X_val.append(subj['data'])
            y_val.append(subj['dx_bin'])
            
    if pca:
        transformer = MiniBatchSparsePCA(n_components=10,
                                         batch_size=3,
                                         random_state=42)
        transformer.fit(X_train)
        X_train = transformer.transform(X_train)
        X_val = transformer.transform(X_val)

    
    return np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val)

## CNN-LSMT

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
class ADHDDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        
        if self.transform:
            x = self.data[index].astype(np.uint8)
            x = self.transform(x)        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [7]:
class Net(nn.Module):
    """
    The idea is the CNN sees bath_size * 47 images independently. Each image has only 1 color channel
    We capture important features from them. To later send for each scan a sequence of 47 features
    to the LSTM to capture the time dependendy between images of the same scan. We take the last state of the LSTM and
    pass it through a couple of FC layers to make a prediction.
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=5, stride=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=5)
        self.lstm = nn.LSTM(input_size=99, hidden_size=10, num_layers=2, batch_first=True) # (batch, seq, feature)
        self.fc1 = nn.Linear(in_features=10, out_features=5)
        self.fc2 = nn.Linear(in_features=5, out_features=2)

    def forward(self, x):
        x = x.view(3*47, 1, 49, 58)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        lstm_out, (ht, ct) = self.lstm(x.view(3, 47, -1))
        x = F.relu(self.fc1(lstm_out[:, -1]))
        x = self.fc2(x)
        
        return x

net = Net()

## Train

Rigth now I'm only doing the training for 1 of the folds.

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
X_train, y_train, X_val, y_val = train_loader(dic, fold, is_reshape=False)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)])

traindataset = ADHDDataset(X_train, list(y_train), transform=transform)
trainloader = DataLoader(traindataset, batch_size=3, drop_last=True)

# get some random training images
dataiter = iter(trainloader)

In [10]:
X_train, y_train, X_val, y_val = train_loader(dic, fold, is_reshape=False)
transform = transforms.Compose(
    [transforms.ToTensor()])
traindataset = ADHDDataset(X_train, list(y_train), transform=transform)
trainloader = DataLoader(traindataset, batch_size=3, drop_last=True)

for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        #inputs = inputs.to(torch.float32)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 0:    # print every 10 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.076
[1,    11] loss: 0.735
[1,    21] loss: 0.739
[1,    31] loss: 0.684
[1,    41] loss: 0.678
[1,    51] loss: 0.699
[2,     1] loss: 0.073
[2,    11] loss: 0.711
[2,    21] loss: 0.710
[2,    31] loss: 0.689
[2,    41] loss: 0.688
[2,    51] loss: 0.694
Finished Training


## Debug NN

In [772]:
conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=5, stride=1)
pool = nn.MaxPool2d(2, 2)
conv2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=5)
lstm = nn.LSTM(input_size=99, hidden_size=10, num_layers=2, batch_first=True)
fc1 = nn.Linear(in_features=10, out_features=5)
fc2 = nn.Linear(in_features=5, out_features=2)

In [784]:
images, labels = next(dataiter)

In [785]:
images.shape

torch.Size([3, 47, 49, 58])

In [786]:
x = images.view(3*47, 1, 49, 58)
x.shape

torch.Size([141, 1, 49, 58])

In [787]:
x = conv1(x)
x.shape

torch.Size([141, 1, 45, 54])

In [788]:
x = pool(F.relu(x))
x.shape

torch.Size([141, 1, 22, 27])

In [789]:
x = conv2(x)
x.shape

torch.Size([141, 1, 18, 23])

In [790]:
x = pool(F.relu(x))
x.shape

torch.Size([141, 1, 9, 11])

In [791]:
x = x.view(3, 47, -1)
x.shape

torch.Size([3, 47, 99])

In [792]:
x, _ = lstm(x)
x.shape

torch.Size([3, 47, 10])

In [793]:
x = x[:, -1]
x.shape

torch.Size([3, 10])

In [794]:
x = F.relu(fc1(x))
x.shape

torch.Size([3, 5])

In [795]:
x = fc2(x)
x.shape

torch.Size([3, 2])

In [797]:
loss = criterion(x, labels)
loss

tensor(0.7220, grad_fn=<NllLossBackward0>)

In [809]:
outputs

tensor([[0.5581, 0.4419],
        [0.5581, 0.4419],
        [0.5581, 0.4419]], grad_fn=<SoftmaxBackward0>)

In [810]:
a, predicted = torch.max(outputs, 1)

In [811]:
a

tensor([0.5581, 0.5581, 0.5581], grad_fn=<MaxBackward0>)

In [812]:
predicted

tensor([0, 0, 0])

In [813]:
labels

tensor([1, 1, 1])